In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ppi = pd.read_csv('../data/PPI/hi-iii.txt', sep = '\t', header = None)

In [3]:
ppi.head(2)

,0,1,2
0,14,64782,1
1,27,9322,1


In [4]:
len(set(ppi[0].tolist() + ppi[1].tolist()))

5604

In [5]:
pd.DataFrame(list(set(ppi[0].tolist() + ppi[1].tolist()))).rename(columns={0: "Protein1_ID"}).to_csv('../../../../scratch/kh2383/IGCN/data/PPI/HuRI/protein_list.csv')

In [6]:
protein_list = pd.DataFrame(list(set(ppi[0].tolist() + ppi[1].tolist()))).rename(columns={0: "Protein1_ID"})

In [7]:
len(ppi)

23322

In [8]:
from itertools import combinations 
comb = combinations(list(protein_list.values.T[0]), 2)

In [9]:
comb = list(comb)

In [10]:
import random

In [11]:
pos = [(i[0], i[1]) for i in (ppi.values)]

In [12]:
neg = list(set(comb) - set(pos))

In [13]:
comb_flipped = [(i[1], i[0]) for i in comb]

In [14]:
neg_2 = list(set(comb_flipped) - set(pos))

In [15]:
neg_2 = [(i[1], i[0]) for i in neg_2]

In [16]:
len(neg)

15681007

In [17]:
neg_final = list(set(neg) & set(neg_2))

In [18]:
len(neg_final)

15676606

In [19]:
len(set(comb))

15699606

In [20]:
random.seed(a = 1)
neg_sample = random.sample(neg_final, len(ppi))

In [21]:
len(neg_sample)

23322

In [22]:
df = pd.DataFrame(pos+neg_sample)
df['label'] = np.array([1]*len(pos) + [0]*len(neg_sample))

In [23]:
df = df.rename({0:'Protein1_ID', 1:'Protein2_ID'}, axis = 1)

In [24]:
def create_fold(df, x):
    test = df.sample(frac = 0.2, replace = False, random_state = x)
    train_val = df[~df.index.isin(test.index)]
    val = train_val.sample(frac = 0.125, replace = False, random_state = 1)
    train = train_val[~train_val.index.isin(val.index)]
    
    path = '../../../../scratch/kh2383/IGCN/data/PPI/HuRI/fold'+str(x)
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')
    test.reset_index(drop = True).to_csv(path + '/test.csv')
    train_val.reset_index(drop = True).to_csv(path + '/train_val.csv')
    
    return train, val, test

In [25]:
train, val, test = create_fold(df, 4)